In [ ]:
markets = [
    'ACB.VN',
    'BCM.VN',
    'BID.VN',
    'BVH.VN',
    'CTG.VN',
    'FPT.VN',
    'GAS.VN',
    'GVR.VN',
    'HDB.VN',
    'HPG.VN',
    'MBB.VN',
    'MSN.VN',
    'MWG.VN',
    'PLX.VN',
    'POW.VN',
    'SAB.VN',
    'SHB.VN',
    'SSB.VN',
    'SSI.VN',
    'STB.VN',
    'TCB.VN',
    'TPB.VN',
    'VCB.VN',
    'VHM.VN',
    'VIB.VN',
    'VIC.VN',
    'VJC.VN',
    'VNM.VN',
    'VPB.VN',
    'VRE.VN',
]

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import yfinance as yf

data = []
for stock in markets:
    data.append(yf.download(stock, start="2023-01-01", end="2024-04-1"))

**HÀM NÀY DÙNG ĐỂ LẤY BIẾN ĐỘNG GIÁ CẢ TRONG KHOẢNG TỪ START ĐẾN END DATE**

Như vậy thời gian giao dịch chứng khoán cơ sở tại Việt Nam là:

Ngày giao dịch: Từ thứ Hai đến thứ Sáu, trong giờ hành chính nhà nước. 

Ngày cuối tuần (thứ Bảy, Chủ Nhật) và ngày lễ tết không tiến hành giao dịch chứng khoán. Các ngày lễ tết không giao dịch chứng khoán ở Việt Nam bao gồm: Tết Dương lịch, Tết Âm lịch, ngày Giỗ tổ Hùng Vương (10/3 Âm lịch), ngày 30/4, ngày 1/5, ngày Quốc khánh 2/9.


Không giao dịch trong những trường hợp hi hữu (như có quy định khẩn cấp hoặc thị trường gặp sự cố).

In [ ]:
import numpy as np
import pandas as pd
def TakeMovement(start_date, end_date):
    movement = []
    for dt in data:
        filtered_data = dt.loc[start_date:end_date]
        open_prices = filtered_data['Open']
        close_prices = filtered_data['Close']
        movement.append(close_prices - open_prices)
    return np.array(movement)

Anh em để 5 ngày thoi

In [ ]:
movement = TakeMovement("2024-03-25", "2024-03-29")

In [ ]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer()

new = normalizer.fit_transform(movement)

print(new.max())
print(new.min())
print(new.mean())

Calculate Calinski-Harabasz score for k from 2-10

In [ ]:
from sklearn.metrics import calinski_harabasz_score
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
x=normalizer.fit_transform(movement)
model = KMeans(max_iter=1000)
visualizer = KElbowVisualizer(
    model, k=(2,11), metric='calinski_harabasz', timings=False,locate_elbow=False
)

visualizer.fit(x)        # Fit the data to the visualizer
visualizer.show()

In [ ]:
from sklearn.pipeline import make_pipeline

k=7

normalizer = Normalizer()

kmeans = KMeans(n_clusters=k, max_iter=1000)

pipeline = make_pipeline(normalizer,kmeans)

In [ ]:
pipeline.fit(movement)

In [ ]:
labels = pipeline.predict(movement)

df = pd.DataFrame({'labels': labels, 'companies': markets})

print(df.sort_values('labels'))

In [ ]:
x=normalizer.fit_transform(movement)
calinski_harabasz_score(x,labels)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
num_clusters = len(df['labels'].unique())
colors = plt.cm.nipy_spectral(np.linspace(0, 1, num_clusters))

for label, color in zip(range(num_clusters), colors):
    subset = df[df['labels'] == label]
    plt.scatter(subset.index, subset['labels'], color=color, label=f'Group {label}', alpha=0.7)

plt.xticks(range(len(df)), df['companies'], rotation=90)
plt.xlabel('', rotation=90)
plt.ylabel('Labels')
plt.title('')
plt.legend(loc='upper right')
plt.grid(True, linestyle='--', alpha=0.5)  # Thêm grid
plt.tight_layout()
plt.show()

In [ ]:
print (calinski_harabasz_score(x,labels))

## Kmedoids

In [ ]:
from sklearn_extra.cluster import KMedoids

### Calculate Calinski-Harabasz for k from 2 to 10

In [ ]:
from yellowbrick.cluster import KElbowVisualizer
x=normalizer.fit_transform(movement)
model = KMedoids(max_iter=1000,init="random")
visualizer = KElbowVisualizer(
    model, k=(2,11), metric='calinski_harabasz', timings=False
)

visualizer.fit(x)        # Fit the data to the visualizer
visualizer.show()

### Kmedoids

In [ ]:
k=3
kmedoids = KMedoids(n_clusters=k, max_iter=1000)

pipeline_1 = make_pipeline(normalizer,kmedoids)

In [ ]:
pipeline_1.fit(movement)

In [ ]:
labels_1 = pipeline_1.predict(movement)

df = pd.DataFrame({'labels': labels_1, 'companies': markets})

print(df.sort_values('labels'))

In [ ]:
plt.figure(figsize=(10, 6))
num_clusters = len(df['labels'].unique())
colors = plt.cm.nipy_spectral(np.linspace(0, 1, num_clusters))

for label, color in zip(range(num_clusters), colors):
    subset = df[df['labels'] == label]
    plt.scatter(subset.index, subset['labels'], color=color, label=f'Group {label}', alpha=0.7)

plt.xticks(range(len(df)), df['companies'], rotation=90)
plt.xlabel('', rotation=90)
plt.ylabel('Labels')
plt.title('')
plt.legend(loc='upper right')
plt.grid(True, linestyle='--', alpha=0.5)  # Thêm grid
plt.tight_layout()
plt.show()